In [ ]:
from bs4 import   BeautifulSoup as bs
import requests as re

In [ ]:
#### Movie Data

. Methods to scrub Disney data

In [ ]:

def get_content_value(row_data):
  if row_data.find('li'):
    return [li.get_text().replace('\xa0',' ') for li in row_data.find_all('li')]

  elif row_data.find('br'):
    return [br for br in row_data.stripped_strings]

  else:
    return row_data.get_text().replace('\xa0',' ')


def remove_tag(soup):
  for tag in soup.find_all(['sup','span']):
    tag.decompose()


def get_info_box(url):
  r=re.get(url)
  soup=bs(r.content)
  info_box=soup.find(class_='infobox vevent')
  info_rows=info_box.find_all('tr')
  
  remove_tag(soup)

  movie_info={}
  for index,row in enumerate(info_rows):
    if index==0 and row.find('th').get_text()!="Planes: Fire & Rescue":
      movie_info['Title']=row.find('th').get_text()
    else:
      header=row.find('th')
      if header:
        content_key=row.find('th').get_text()
        content_value=get_content_value(row.find('td'))
        movie_info[content_key]=content_value
  return movie_info

. Scrubing Disney movie data

In [ ]:
r=re.get('https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films')
path='https://en.wikipedia.org/'
soup=bs(r.content)
movies=soup.select('.wikitable.sortable i a')

movie_info_list=[]
for index,movie in enumerate(movies):
  try:
    relative_path=movie['href']
    full_path=path + relative_path
    title=movie['title']
    movie_info_list.append(get_info_box(full_path))
  except Exception as e:
    print(movie.get_text())
    print(e)


. Cleaning and converting running time into mins

In [ ]:
#### Methods to convert running time into int datatype

def mins_to_intiger (running_time):
  try:
    if running_time=='N/A':
      return None
    

    if isinstance(running_time,list):
      return int(running_time[0].split(' ')[0])

    # elif running_time.isalnum():
    #   return [int(s) for s in running_time.split() if s.isdigit()]
    
    else:
      return int(running_time.split(' ')[0])
    
  except Exception as e:
    print(e)



In [ ]:
### converting mins into intiger

for movie in movie_info_list:
  movie['Running time (mins)']=mins_to_intiger(movie.get('Running time','N/A'))

. Usiing reguler expression, converting Boxoffice and Budget into numeric value

In [ ]:
import re

In [ ]:
#### Methods for Budget & Box office data conversion

amounts=r'thousand|million|billion'
pattern= r'\d+(,\d{3})*\.*\d*'
value_re=rf'\${pattern}'
word_re=rf'\${pattern}(-|\sto\s|-)?({pattern})?\s({amounts})'


def word_to_digit(word):
  digit_dict={'thousand':1000,'million':1000000,'billion':1000000000}
  return digit_dict[word]

def parse_word_syntax(string):
  value_str=re.search(pattern,string).group()
  value=float(value_str.replace(',',''))
  word_str=re.search(amounts,string,flags=re.I).group().lower()
  word_value=word_to_digit(word_str)
  return word_value * value


def parse_value_syntax(string):
  value_str=re.search(pattern,string).group()
  value=float(value_str.replace(',',''))
  return value



def money_conversion(money):
  if money=='N/A':
    return None
  if isinstance(money,list):
    money=money[0]

  word_syntax=re.search(word_re,money,flags=re.I)
  value_syntax=re.search(value_re,money)

  if word_syntax:
    return parse_word_syntax(word_syntax.group())

  elif value_syntax:
    return parse_value_syntax(value_syntax.group())
  else:
    return None

In [ ]:
#### Budget & Box office data conversion

for movie in movie_info_list:
  movie['Budget (float)']=money_conversion(movie.get('Budget','N/A'))
  movie['Box office (float)']=money_conversion(movie.get('Box office','N/A'))


In [ ]:
#### convert date into datetime

. Converting Release date data into datetime format/obj

In [ ]:
#### Methods for datetime conversion 

from datetime import datetime
dates=[movie.get('Release date','N/A') for movie in movie_info_list]


def clean_date(date):
  return date.split('(')[0].strip()

def date_conversion(date):
  if isinstance(date,list):
    date= date[0]

  if date=='N/A':
    return None

  date_str=clean_date(date)
  frmts=['%B %d, %Y','%d %B %Y','%B %Y','%Y']

  for frmt in frmts:
    try:
      return datetime.strptime(date_str,frmt)
    except:
      pass
  return None


In [ ]:
#### converting dates into datetime
for movie in movie_info_list:
  movie['Release date (datetime)']=date_conversion(movie.get('Release date','N/A'))

In [ ]:
# for movie in movie_info_list:
#    print(movie.get('Release date (datetime)'),movie.get('Release date'))

. Finally storing the dateset into different file/format

In [ ]:
#### Method to save data into a file

import json
def save_data(file_name, data):
  try:
    with open(file_name,'w',encoding='utf-8') as f:
      json.dump(data,f,ensure_ascii=False,indent=2,default=str)
  except Exception as e:
    print(e)

In [ ]:
#### Load JSON file

import json

def load_data(file_name):
  with open(file_name,encoding='utf-8') as f:
    return file_name

In [ ]:
#### dumping JSON

save_data('disney_json_data.json',movie_info_list)

In [ ]:
#### Method to dump pickle data

import pickle
def dump_data_pickle(file,data):
  with open(file,'wb') as f:
    pickle.dump(data,f)

In [ ]:
#### Method to load pickle data

import pickle
def load_data_pickle(file):
  with open(file,'rb') as f:
    return pickle.load(f)

In [ ]:
# Saving pickle data
dump_data_pickle('Disney_Pickle_Data.pickle',movie_info_list)

In [ ]:
#### Loading Pickle data

df=load_data_pickle('Disney_Pickle_Data.pickle')

In [ ]:
#### Creating DataFrame

import pandas as pd
df=pd.DataFrame(movie_info_list)


In [ ]:
#### Writing data into CSV file

df.to_csv('Disney_csv_Data.csv')